In [17]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import re
import fnmatch
import matplotlib.pyplot as plt
from functools import reduce
from bs4 import BeautifulSoup
import requests
import osmnx as ox
from API_KEY import get_OneMap_token
import networkx as nx

# Proximity to primary schools
- OneMap's SchoolQuery and [MOE's SchoolFinder](https://www.moe.gov.sg/schoolfinder) uses 1km and between 1-2km as thresholds to determine parent's priority in getting their child placement in a primary school.
- The distance is not measured using euclidean distance, it is measured using SLA's OneMap SchoolQuery or MOE Distance Checker Tool.
- Distance is calculated using the shortest path along the road network and footpaths.
- As per OneMap's visualisation, I can confirm that the distance is not measured using euclidean distance

In [18]:
save_dir = os.path.join(os.getcwd(),"Data","Road_Networks")

# import car network
filename = "SG_car_network.graphml"
G_car = ox.load_graphml(os.path.join(save_dir,filename))

# import walking network
filename = "SG_walk_network.graphml"
G_walk = ox.load_graphml(os.path.join(save_dir, filename))

# Scrap top primary schools ranked from best to worst

In [19]:
URL = "https://www.creativecampus.com.sg/best-primary-schools-in-singapore-2024"
page = requests.get(URL)
# parse HTML
soup = BeautifulSoup(page.content, "html.parser")

# Find the table
table = soup.find('table', {'id': 'primary'})

# Extract all school names (first <td> of each <tr> in <tbody>)
school_names = [row.find_all('td')[0].text.strip() for row in table.find('tbody').find_all('tr')]

# Output the list
print(school_names)



["Methodist Girls' School (Primary)", 'Tao Nan School', 'Ai Tong School', "Holy Innocents' Primary School", "CHIJ St. Nicholas Girls' School", 'Admiralty Primary School', "St. Joseph's Institution Junior", 'Catholic High School', 'Anglo-Chinese School (Junior)', 'Chongfu School', 'Kong Hwa School', "St. Hilda's Primary School", 'Anglo-Chinese School (Primary)', 'Nan Chiau Primary School', 'Nan Hua Primary School', 'Nanyang Primary School', 'Pei Hwa Presbyterian Primary School', 'Kuo Chuan Presbyterian Primary School', 'Rulang Primary School', "Singapore Chinese Girls' Primary School", 'Maris Stella High School', 'Fairfield Methodist School (Primary)', 'South View Primary School', 'CHIJ Primary (Toa Payoh)', 'Henry Park Primary School', 'Pei Chun Public School', 'Xinmin Primary School', 'Red Swastika School', "St. Anthony's Primary School", 'Rosyth School', "Paya Lebar Methodist Girls' School (Primary)", 'Northland Primary School', 'Pasir Ris Primary School', 'Mee Toh School', 'Radin Ma

## Retrieve corresponding coordinates of primary school

In [20]:
def generate_OneMap_headers():
    """ generates new one map token """
    onemapKey = get_OneMap_token()
    headers = {"Authorization": onemapKey}
    return headers

def get_coordinates_from_location(location,headers):
    """returns number of results found, search value, and coordinates given a supplied location 
    Args:
        location (str): a location in singapore
    Returns:
        tuple: strings corresponding to number of results found, search value, lat, and lon
    """
    
    url = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={location}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
        
    response = requests.request("GET", url, headers=headers)
    response = response.json()
    response_first_result = response['results'][0] # get first item in the list
    return response_first_result

headers = generate_OneMap_headers()
searchVal = get_coordinates_from_location(school_names[0],headers=headers)
searchVal

{'SEARCHVAL': "PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY)",
 'BLK_NO': '298',
 'ROAD_NAME': 'LORONG AH SOO',
 'BUILDING': "PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY)",
 'ADDRESS': "298 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY) SINGAPORE 536741",
 'POSTAL': '536741',
 'X': '33748.3760953182',
 'Y': '36843.4187818952',
 'LATITUDE': '1.34947276268396',
 'LONGITUDE': '103.884971343143'}

## Convert to df

In [21]:
school_list = []
for s in school_names:
    searchVal = get_coordinates_from_location(s,headers=headers)
    searchVal['School Name'] = s
    school_list.append(searchVal)

school_df = pd.DataFrame(school_list)
school_df.head()

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,School Name
0,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),298,LORONG AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),298 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,536741,33748.3760953182,36843.4187818952,1.34947276268396,103.884971343143,Methodist Girls' School (Primary)
1,FORMER TAO NAN SCHOOL,39,ARMENIAN STREET,FORMER TAO NAN SCHOOL,39 ARMENIAN STREET FORMER TAO NAN SCHOOL SINGA...,179941,29748.116507777,30746.1923441123,1.2943321052493,103.849026157822,Tao Nan School
2,AI TONG SCHOOL,100,BRIGHT HILL DRIVE,AI TONG SCHOOL,100 BRIGHT HILL DRIVE AI TONG SCHOOL SINGAPORE...,579646,27966.8088298785,38071.9191181567,1.3605834338904,103.833020333986,Ai Tong School
3,HOLY INNOCENTS' PRIMARY SCHOOL,5,LORONG LOW KOON,HOLY INNOCENTS' PRIMARY SCHOOL,5 LORONG LOW KOON HOLY INNOCENTS' PRIMARY SCHO...,536451,34765.9036816582,38774.6943912152,1.36693830877349,103.894114899795,Holy Innocents' Primary School
4,CHIJ SAINT NICHOLAS GIRLS' SCHOOL,501,ANG MO KIO STREET 13,CHIJ SAINT NICHOLAS GIRLS' SCHOOL,501 ANG MO KIO STREET 13 CHIJ SAINT NICHOLAS G...,569405,28104.0195931299,39497.6106162649,1.37347687878916,103.834253269436,CHIJ St. Nicholas Girls' School


### Get nodesID for school

In [22]:
school_df[['LONGITUDE','LATITUDE']] = school_df[['LONGITUDE','LATITUDE']].apply(pd.to_numeric, errors='coerce')
school_df['nodesID_school_car'] = ox.nearest_nodes(G_car,X=school_df['LONGITUDE'], Y=school_df['LATITUDE'])
school_df['nodesID_school_walk'] = ox.nearest_nodes(G_walk,X=school_df['LONGITUDE'], Y=school_df['LATITUDE'])

school_df.head()

,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,School Name,nodesID_school_car,nodesID_school_walk
0,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),298,LORONG AH SOO,PAYA LEBAR METHODIST GIRLS' SCHOOL (PRIMARY),298 LORONG AH SOO PAYA LEBAR METHODIST GIRLS' ...,536741,33748.3760953182,36843.4187818952,1.349473,103.884971,Methodist Girls' School (Primary),366226693,366226693
1,FORMER TAO NAN SCHOOL,39,ARMENIAN STREET,FORMER TAO NAN SCHOOL,39 ARMENIAN STREET FORMER TAO NAN SCHOOL SINGA...,179941,29748.116507777,30746.1923441123,1.294332,103.849026,Tao Nan School,7152969482,6673357533
2,AI TONG SCHOOL,100,BRIGHT HILL DRIVE,AI TONG SCHOOL,100 BRIGHT HILL DRIVE AI TONG SCHOOL SINGAPORE...,579646,27966.8088298785,38071.9191181567,1.360583,103.833020,Ai Tong School,4598816959,5270039725
3,HOLY INNOCENTS' PRIMARY SCHOOL,5,LORONG LOW KOON,HOLY INNOCENTS' PRIMARY SCHOOL,5 LORONG LOW KOON HOLY INNOCENTS' PRIMARY SCHO...,536451,34765.9036816582,38774.6943912152,1.366938,103.894115,Holy Innocents' Primary School,2959762943,5214893587
4,CHIJ SAINT NICHOLAS GIRLS' SCHOOL,501,ANG MO KIO STREET 13,CHIJ SAINT NICHOLAS GIRLS' SCHOOL,501 ANG MO KIO STREET 13 CHIJ SAINT NICHOLAS G...,569405,28104.0195931299,39497.6106162649,1.373477,103.834253,CHIJ St. Nicholas Girls' School,5809936563,8079538307


### Export as csv

In [23]:
school_df.to_csv(os.path.join(os.getcwd(),"Data","topPrimarySchools.csv"),index=False)